In [1]:
from EM_part import *

In [2]:
args_result = np.load('./numpy/arg_all_result041201.npy',allow_pickle=True).tolist()

In [3]:
args_result

[[0.56,
  array([[ 0.13848921],
         [ 0.09714998],
         [ 0.17688305],
         [ 0.04271415],
         [ 0.26635781],
         [-0.0329182 ]]),
  [array([[0.76710826],
          [0.05062531],
          [4.67134601]]),
   array([[ 0.14396583, -0.07979323, -0.29058896],
          [-0.07979323,  0.2703215 ,  0.24254395],
          [-0.29058896,  0.24254395,  1.77014481]]),
   0.06595832023554629],
  [array([[ 0.7986125 ],
          [-0.015506  ],
          [ 4.35970845]]),
   array([[ 0.12037906, -0.01952636, -0.44914114],
          [-0.01952636,  0.24281368, -0.10533364],
          [-0.44914114, -0.10533364,  3.04472004]]),
   0.06485961631249063],
  [array([[ 0.56735183],
          [-0.09888943],
          [ 6.31679006]]),
   array([[ 0.1197014 , -0.07407517, -0.37007806],
          [-0.07407517,  0.2002001 ,  0.11488745],
          [-0.37007806,  0.11488745,  4.84808139]]),
   0.04469373229168624],
  [array([[-0.43623345],
          [-0.14347801],
          [-6.30203158]]),
 

In [4]:
# for i in range(100):
#     print(get_hat_rho_l(test_dataset[i],*args_result))
mode_predict_result = [get_hat_rho_l(test_dataset[i],*args_result).reshape(2) for i in range(100)]
mode_true = [test_dataset[i]["mode"] for i in range(100)]

In [5]:
mode_predict_result

[array([0., 1.]),
 array([1., 0.]),
 array([0., 1.]),
 array([1., 0.]),
 array([1., 0.]),
 array([1.00000000e+00, 3.07518759e-39]),
 array([1., 0.]),
 array([1., 0.]),
 array([1., 0.]),
 array([1., 0.]),
 array([1., 0.]),
 array([1., 0.]),
 array([1., 0.]),
 array([1., 0.]),
 array([0., 1.]),
 array([0., 1.]),
 array([0., 1.]),
 array([0., 1.]),
 array([0., 1.]),
 array([0., 1.]),
 array([0., 1.]),
 array([0., 1.]),
 array([0., 1.]),
 array([0., 1.]),
 array([1., 0.]),
 array([1., 0.]),
 array([0.30722052, 0.69277948]),
 array([0., 1.]),
 array([1., 0.]),
 array([0., 1.]),
 array([0., 1.]),
 array([1., 0.]),
 array([1., 0.]),
 array([1., 0.]),
 array([0., 1.]),
 array([1.00000000e+00, 4.80722071e-10]),
 array([7.32076993e-24, 1.00000000e+00]),
 array([1., 0.]),
 array([0., 1.]),
 array([0., 1.]),
 array([0., 1.]),
 array([1., 0.]),
 array([0., 1.]),
 array([1., 0.]),
 array([0., 1.]),
 array([0., 1.]),
 array([1.0000000e+00, 8.8152978e-33]),
 array([1., 0.]),
 array([0., 1.]),
 array([

In [7]:
corr = 0
err = 0
for i in range(100):
    if np.sum(mode_predict_result[i] - mode_true [i]) < 0.005:
        corr+=1
    else:
        err+=1
print(corr,err)
    

100 0


In [8]:
def mode_para_selection(instance_q,dk_list,*args):
    """
    *args [pi,w,[mu,sigma],[mu,sigma]],[pi,w,[mu,sigma],[mu,sigma]]
    return [pi,w,mu,...]
    """
    mode_q = np.argmax(get_hat_rho_l(instance_q,*args))
    selected_args = []
    args_q = args[mode_q]
    selected_args.append(dk_list[mode_q]) # dk
    selected_args.append(args_q[1]) # wk
    #selected_args.appned(args_q[1])
    for idx in range(3):
        selected_args.append(args_q[-1][idx])
    return selected_args

select_args = mode_para_selection(test_dataset[0],[1,1], *args_result)   

In [9]:
def get_post_Sigma_q_and_mu_q(instance_q,*args):
    """
    args:select_args,[hat_D,w,mu,...]
    """
    [hat_D,w,mu_Gamma,C_Gamma,sigma_2] = args
    x_q = instance_q["input"]
    Phi_q = instance_q["Phi_l"]
    Sigma_q = np.linalg.inv((Phi_q.T @ Phi_q) / sigma_2 + np.linalg.inv(C_Gamma))
    mu_q = Sigma_q @ ((Phi_q.T @ x_q @ w) / sigma_2 + np.linalg.inv(C_Gamma) @ mu_Gamma)
    return mu_q, Sigma_q
mu_q,Sigma_q = get_post_Sigma_q_and_mu_q(test_dataset[0],*select_args)

In [10]:
import scipy

In [11]:
def get_CDF_t(instance_q,t,*args):
    n_q = len(instance_q["input"])
    Phi_q = instance_q["Phi_l"]
    [hat_D,w,mu_Gamma,C_Gamma,sigma_2] = args
    t_nl_t = n_q + t
    Phi_t_nl_t = np.array([1, t_nl_t / 500, (t_nl_t / 500) * (t_nl_t / 500)], 
                          dtype=np.float64).reshape(1,3)
    mu_q, Sigma_q = get_post_Sigma_q_and_mu_q(instance_q,*args)
    mu_t_nl_t_0 = Phi_q[-1] @ mu_q
    mu_t_nl_t = Phi_t_nl_t @ mu_q
    sigma_t_nl_t = Phi_t_nl_t @ Sigma_q @ Phi_t_nl_t.T
    if mu_t_nl_t_0<hat_D:
        A = ((hat_D - mu_t_nl_t) / (np.sqrt(sigma_t_nl_t))).item()
    else :
        A = ((-hat_D + mu_t_nl_t) / (np.sqrt(sigma_t_nl_t))).item()
    Phi_t = 1 - scipy.stats.norm.cdf(A)
    return Phi_t

get_CDF_t(test_dataset[0],69, *select_args)    
    

0.4551764720681437

In [30]:
def get_single_RUL(instance_q,*args):
    """
    # *args:[pi,w,[mu,sigma],[mu,sigma]],[pi,w,[mu,sigma],[mu,sigma]] previous results
    select_args [hat_D,w,mu,...]
    """
    # select_args = mode_para_selection(instance_q,dk_list, *args)
    box = []
    for t in range(500):
        A = get_CDF_t(instance_q,t,*args)
        B = get_CDF_t(instance_q,0,*args)
        C = (A-B)/(1-B)
        D = abs(C-0.5)
        box.append(D)
    RUL_q = box.index(min(box)) + 1
    return RUL_q

get_single_RUL(test_dataset[0],*select_args)
    

71

In [31]:
def get_RUL(instance, dk_list, *args):
    """
    *args:[pi,w,[mu,sigma],[mu,sigma]],[pi,w,[mu,sigma],[mu,sigma]] previous results
    """
    length = len(instance)
    RUL = []
    for idx in tqdm(range(length)):
        select_args = mode_para_selection(instance[idx],dk_list, *args)
        RUL_q = get_single_RUL(instance[idx], *select_args)
        RUL.append(RUL_q)
    return np.array(RUL)

In [32]:
get_RUL(test_dataset,[1,1],*args_result)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:50<00:00,  1.99it/s]


array([ 71,  54,  84, 108, 500, 119,  55,  56,  56,  87,  76,  94, 106,
        39, 231,  94, 500, 500, 216, 135,  23, 297, 256,   8,  50,   9,
       139, 250, 108, 212, 303,  50,  81,  68, 166, 155, 225,  27,   1,
        88, 246, 112, 335,  60, 268,  13, 122,  76,   1,  16,  88, 155,
        66, 141,  56, 116, 111,  43,  49, 115, 269, 189, 322,  77, 228,
       104, 221, 500, 298,  82,  79,  62, 213, 342,  74,  19, 167,  41,
        22, 500,  38,  11, 165,  84, 103,  36, 150, 149,  99,  53, 115,
        23, 163,   5, 329, 500, 242,  24,   5,  91])

In [16]:
#RUL
path = DATA_PATH+'RUL.csv'
RUL_frame = pd.read_csv(path, header=None)
RUL = RUL_frame.values[:, 0]
RUL

array([ 44,  51,  27, 120, 101,  99,  71,  55,  55,  66,  77, 115, 115,
        31, 108,  56, 136, 132,  85,  56,  18, 119,  78,   9,  58,  11,
        88, 144, 124,  89,  79,  55,  71,  65,  87, 137, 145,  22,   8,
        41, 131, 115, 128,  69, 111,   7, 137,  55, 135,  11,  78, 120,
        87,  87,  55,  93,  88,  40,  49, 128, 129,  58, 117,  28, 115,
        87,  92, 103, 100,  63,  35,  45,  99, 117,  45,  27,  86,  20,
        18, 133,  15,   6, 145, 104,  56,  25,  68, 144,  41,  51,  81,
        14,  67,  10, 127, 113, 123,  17,   8,  28], dtype=int64)

In [66]:
args = args_result
instance_q = test_dataset[1]
dk_list = [-1,1]
select_args = mode_para_selection(instance_q,dk_list, *args)
# get_post_Sigma_q_and_mu_q(instance_q,*select_args)
n_q = len(instance_q["input"])
[hat_D,w,mu_Gamma,C_Gamma,sigma_2] = select_args
t = 0
t_nl_t = n_q + t
Phi_t_nl_t = np.array([1, t_nl_t / 500, (t_nl_t / 500) * (t_nl_t / 500)], 
                      dtype=np.float64).reshape(1,3)
mu_q, Sigma_q = get_post_Sigma_q_and_mu_q(instance_q,*select_args)
mu_t_nl_t = Phi_t_nl_t @ mu_q
sigma_t_nl_t = Phi_t_nl_t @ Sigma_q @ Phi_t_nl_t.T
if mu_t_nl_t_0<hat_D:
    A = ((hat_D - mu_t_nl_t) / (np.sqrt(sigma_t_nl_t))).item()
else :
    A = ((-hat_D + mu_t_nl_t) / (np.sqrt(sigma_t_nl_t))).item()
Phi_t = 1 - scipy.stats.norm.cdf(A)
# get_CDF_t(instance_q,0,*select_args)


In [68]:
mu_t_nl_t,Phi_t 

(array([[-0.68126908]]), 1.0)

In [61]:
args 

[[0.56,
  array([[-0.14107795],
         [-0.07269193],
         [-0.1833172 ],
         [-0.0283842 ],
         [-0.28645507],
         [ 0.01634209]]),
  [array([[ 0.80116542],
          [-0.49698436],
          [ 6.19754031]]),
   array([[ 0.13931929, -0.03756375, -0.36207992],
          [-0.03756375,  0.14939987, -0.28534003],
          [-0.36207992, -0.28534003,  6.09838779]]),
   0.0655713138859101],
  [array([[ 0.82468967],
          [-0.42851781],
          [ 5.50735066]]),
   array([[ 0.11349203,  0.03677791, -0.56846498],
          [ 0.03677791,  0.16090321, -0.50379678],
          [-0.56846498, -0.50379678,  6.20848893]]),
   0.0646515251422192],
  [array([[ 0.58496356],
          [-0.37879269],
          [ 7.09347277]]),
   array([[ 0.11858449, -0.06840603, -0.35377243],
          [-0.06840603,  0.2823017 , -0.6051624 ],
          [-0.35377243, -0.6051624 ,  8.35371785]]),
   0.04456603529665039],
  [array([[-0.46930976],
          [ 0.39135126],
          [-7.80706822]]),
